In [1]:
import pandas as pd
import numpy as np 
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW
from tqdm import tqdm 

In [2]:
train = pd.read_csv("train.csv")

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-1.3b", eos_token="</s>")

formatted_data = [] 
for _, row in tqdm(train.iterrows()):
    for q_col in ["질문_1", "질문_2"]:
        for a_col in ["답변_1", "답변_2", "답변_3", "답변_4", "답변_5"]:
            # 질문과 답변 쌍을 </s> token으로 연결 
            input_text = row[q_col] + tokenizer.eos_token + row[a_col] 
            input_ids = tokenizer.encode(input_text, return_tensors="pt")
            formatted_data.append(input_ids)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
644it [00:01, 362.24it/s]


In [3]:
device = torch.device("cuda") 

model = AutoModelForCausalLM.from_pretrained("EleutherAI/polyglot-ko-1.3b")
model.to(device) 

model

print() 

In [ ]:
CFG = {
    "LR": 2e-5, 
    "EPOCHS": 1, 
}

optimizer = AdamW(model.parameters(), lr=CFG['LR'])
model.train()

for epoch in range(CFG['EPOCHS']):
    total_loss = 0
    progress_bar = tqdm(enumerate(formatted_data), total=len(formatted_data))
    for batch_idx, batch in progress_bar:
        # 데이터를 GPU단으로 이동
        batch = batch.to(device)
        outputs = model(batch, labels=batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        # 진행률 표시줄에 평균 손실 업데이트
        progress_bar.set_description(f"Epoch {epoch+1} - Avg Loss: {total_loss / (batch_idx+1):.4f}")

    # 에폭의 평균 손실을 출력
    print(f"Epoch {epoch+1}/{CFG['EPOCHS']}, Average Loss: {total_loss / len(formatted_data)}")

# 모델 저장
model_save_path = "./hansol-polyglot1.3b.pth"
torch.save(model.state_dict(), model_save_path)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1 - Avg Loss: 1.3771:  28%|██▊       | 1830/6440 [04:47<11:49,  6.50it/s]

In [ ]:
# 저장된 Fine-tuned 모델과 토크나이저 불러오기
model.load_state_dict(torch.load(model_save_path))
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-1.3b", eos_token="</s>")

# Inference를 위한 test.csv 파일 로드
test = pd.read_csv('./test.csv')

# test.csv의 '질문'에 대한 '답변'을 저장할 리스트
preds = []

# '질문' 컬럼의 각 질문에 대해 답변 생성
for test_question in tqdm(test['질문']):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    output_sequences = model.generate(
        input_ids=input_ids.to(device),
        max_length=300,
        temperature=0.9,
        top_k=1,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1
    )

    # 생성된 텍스트(답변) 저장
    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        # 질문과 답변의 사이를 나타내는 eos_token (</s>)를 찾아, 이후부터 출력
        answer_start = full_text.find(tokenizer.eos_token) + len(tokenizer.eos_token)
        answer_only = full_text[answer_start:].strip()
        answer_only = answer_only.replace('\n', ' ')
        preds.append(answer_only)

In [ ]:
preds[0]

In [ ]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(preds)
pred_embeddings.shape
submit = pd.read_csv('./sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()
# 리더보드 제출을 위한 csv파일 생성
submit.to_csv('./polyglot1.3b_baseline_v2.csv', index=False)
